# Script #5 - Plotly Data Visualizer

*Mike Huff, 2025*

https://github.com/m-huff

This script visualizes your GEOTIFF raster data as an interactive heatmap using **Plotly**, a cool Python library that produces HTML-based data visualizations that are actually interactive! It reads a GeoTIFF file containing lightning optical energy values and plots them in a visually appealing “earthy” color scheme that transitions from deep green to gold to red, representing increasing energy intensity. After loading the raster with **rasterio**, the script extracts coordinate bounds and builds x/y axes for spatial reference. The custom `plot_raster()` function uses **Plotly’s Heatmap** to render the data with a dark background optimized for contrast and presentation. The output is saved as an interactive HTML file (`glm_flash_energy_visualization.html`) that opens automatically in a web browser, allowing users to pan, zoom, and inspect lightning energy patterns across the GOES domain. The HTML file is automatically output into the folder your input GEOTIFF is in.

An example HTML output from this script is included in this GitHub repository.

**How to use:**

1. Update `GEOTIFF_PATH` to your own GeoTIFF file.
2. Change `OUTPUT_HTML_FILE_NAME` if desired.
3. An HTML visualization of the GOES lightning energy field will open automatically in your default browser.


In [ ]:
%pip install plotly numpy rasterio

import numpy as np
import plotly.graph_objects as go
import rasterio
from rasterio.transform import Affine
from plotly.offline import plot

### VARIABLES TO CONTROL THE SCRIPT
### THESE ARE DESCRIBED IN THE MARKDOWN CELL ABOVE
GEOTIFF_PATH = "E:/GOES-R Lightning Data/EAST-RASTERS/east_mean_energy_2025_cleaned.tif"
OUTPUT_HTML_FILE_NAME = 'glm_flash_energy_visualization.html'

# EARTHY COLOR RAMP - GREEN/GOLD/RED
CUSTOM_COLORSCALE = [
    [0.0, 'rgb(12, 20, 12)'],
    [0.25, 'rgb(38, 51, 59)'],
    [0.50, 'rgb(56, 75, 73)'],
    [0.75, 'rgb(192, 139, 87)'],
    [1, 'rgb(152, 91, 54)']
]

def plot_raster(data, x_coords, y_coords, transform):

    trace = go.Heatmap(
        z=data, 
        x=x_coords, 
        y=y_coords,
        colorscale=CUSTOM_COLORSCALE,
        reversescale=False,
        zmin=0,
        zmax=np.percentile(data, 99.5),
        colorbar=dict(
            title='',
            len=0.5,
            x=1.05,
            thickness=20,
        )
    )

    layout = go.Layout(
        title={
            'text': '',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 24, 'color': 'white'}
        },

        paper_bgcolor='black',
        plot_bgcolor='black',
        autosize=True,
        hovermode='closest',
        
        xaxis=dict(
            title='Longitude',
            showgrid=False,
            zeroline=False,
            color='gray',
            range=[x_coords.min(), x_coords.max()]
        ),
        yaxis=dict(
            title='Latitude',
            showgrid=False,
            zeroline=False,
            color='gray',
            range=[y_coords.min(), y_coords.max()]
        ),
        margin=dict(l=50, r=50, t=50, b=50)
    )

    fig = go.Figure(data=[trace], layout=layout)
    
    plot(fig, filename=OUTPUT_HTML_FILE_NAME, auto_open=True)

def main():

    try:
        with rasterio.open(GEOTIFF_PATH) as src:
            energy_data = src.read(1)
            transform = src.transform

            x_coords = np.linspace(src.bounds.left, src.bounds.right, src.width)
            y_coords = np.linspace(src.bounds.top, src.bounds.bottom, src.height)

            print(f"--- Loaded Raster Data from {GEOTIFF_PATH} ---")
            print(f"Shape: {energy_data.shape}, Min/Max Energy: {energy_data.min():.2f} J / {energy_data.max():.2f} J")

    except Exception as e:
        print(f"An unexpected error occurred while reading the GeoTIFF: {e}")

    plot_raster(energy_data, x_coords, y_coords, transform)

if __name__ == '__main__':
    main()

